<a href="https://colab.research.google.com/github/MollaMohammadIbrahim/MachineLearning/blob/main/digitDetectionUsungTransferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importing libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load the MNIST dataset

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


# Preprocess the data

In [ ]:
x_train = np.pad(x_train, ((0,0),(10,10),(10,10)))
print(x_train.shape)
x_test = np.pad(x_test, ((0,0),(10,10),(10,10)))
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
#used to repeat the 1 channel of the input images to create 3 channels,so that the images can be used
#with pre-trained models like VGG16 that are trained on RGB images.

(60000, 48, 48)


# Define the model

In [ ]:
base_model = VGG16(include_top=False, pooling='avg', weights='imagenet', input_shape=(48, 48, 3))
for layer in base_model.layers:
    layer.trainable = False
model = Sequential([
    base_model,
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


# Compile the model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
469/469 [==============================] - 23s 40ms/step - loss: 0.5541 - accuracy: 0.8408 - val_loss: 0.2364 - val_accuracy: 0.9329
Epoch 2/10
469/469 [==============================] - 17s 37ms/step - loss: 0.2123 - accuracy: 0.9369 - val_loss: 0.1598 - val_accuracy: 0.9551
Epoch 3/10
469/469 [==============================] - 18s 38ms/step - loss: 0.1632 - accuracy: 0.9503 - val_loss: 0.1380 - val_accuracy: 0.9582
Epoch 4/10
469/469 [==============================] - 17s 37ms/step - loss: 0.1417 - accuracy: 0.9565 - val_loss: 0.1201 - val_accuracy: 0.9631
Epoch 5/10
469/469 [==============================] - 17s 36ms/step - loss: 0.1287 - accuracy: 0.9597 - val_loss: 0.1105 - val_accuracy: 0.9668
Epoch 6/10
469/469 [==============================] - 17s 37ms/step - loss: 0.1176 - accuracy: 0.9633 - val_loss: 0.1071 - val_accuracy: 0.9670
Epoch 7/10
469/469 [==============================] - 17s 37ms/step - loss: 0.1123 - accuracy: 0.9644 - val_loss: 0.1091 - val_accuracy:

# Evaluate the model on the test set

In [ ]:

loss, accuracy = model.evaluate(x_test, y_test)
print('Test accuracy:', accuracy)


313/313 [==============================] - 4s 12ms/step - loss: 0.1039 - accuracy: 0.9664
Test accuracy: 0.9664000272750854
